<a href="https://colab.research.google.com/github/ronniesong0809/chestX-ray/blob/master/chestX_ray.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
!echo '{"username":"ronniesong0809","key":"269788090750b1eb7d55a04d86886708"}' > kaggle.json
!cp /content/kaggle.json /root/.kaggle/kaggle.json

In [0]:
!cp /content/kaggle.json /root/.kaggle/kaggle.json
!chmod 600 /root/.kaggle/kaggle.json
!kaggle config set -n path -v/content

In [0]:
!kaggle datasets download -d paultimothymooney/chest-xray-pneumonia

In [0]:
!unzip datasets/paultimothymooney/chest-xray-pneumonia/chest-xray-pneumonia.zip

In [0]:
import pandas as pd
import numpy as np
import os
import matplotlib.pyplot as plt
%matplotlib inline

In [0]:
import keras
from keras.models import Sequential
from keras.layers import Conv2D,MaxPool2D,Dense,Dropout,Softmax,Input,Flatten, SeparableConv2D
from keras.optimizers import Adam,RMSprop,SGD
from keras.layers.merge import add
from keras.layers import Dense, Activation, Flatten
from keras.layers import Conv2D, MaxPooling2D, BatchNormalization
from keras.layers import BatchNormalization
from keras.metrics import categorical_accuracy
from keras.preprocessing.image import ImageDataGenerator

from sklearn.metrics import roc_auc_score,roc_curve,accuracy_score,recall_score

# from tensorflow import set_random_seed
import tensorflow as tf

In [0]:
os.environ['PYTHONHASHSEED'] = "0"
np.random.seed(1)
# set_random_seed(2)
tf.random.set_seed(2)

## Model 1: Simple CNN

In [0]:
model = Sequential()
model.add(Conv2D(filters=32, kernel_size=(3,3), activation="relu", padding="same", input_shape=(64,64,1)))
model.add(Conv2D(filters=32, kernel_size=(3,3), activation="relu", padding="same"))
model.add(BatchNormalization())
model.add(MaxPooling2D(pool_size=(2,2)))
model.add(Dropout(rate=0.25))

model.add(Conv2D(filters=64, kernel_size=(3,3), activation="relu", padding="same"))
model.add(Conv2D(filters=64, kernel_size=(3,3), activation="relu", padding="same"))
model.add(BatchNormalization())
model.add(MaxPooling2D(pool_size=(2,2)))
model.add(Dropout(rate=0.25))

model.add(Flatten())
model.add(Dense(1024,activation="relu"))
model.add(BatchNormalization())
model.add(Dropout(rate=0.4))
model.add(Dense(2, activation="softmax"))

## Model 2: VGG16

In [0]:
# from keras.layers import SeparableConv2D, ZeroPadding2D

# model = Sequential()
# model.add(Conv2D(64, (3,3), activation='relu', padding='same', name='Conv1_1', input_shape=(224,224,1)))
# model.add(Conv2D(64, (3,3), activation="relu", padding="same"))
# # model.add(BatchNormalization())
# model.add(MaxPooling2D(pool_size=(2, 2), strides=(2, 2), padding="SAME"))

# model.add(SeparableConv2D(128, (3,3), activation='relu', padding='same'))
# model.add(SeparableConv2D(128, (3,3), activation='relu', padding='same'))
# # model.add(BatchNormalization())
# model.add(MaxPooling2D(pool_size=(2, 2), strides=(2, 2), padding="SAME"))

# model.add(SeparableConv2D(256, (3,3), activation='relu', padding='same'))
# model.add(SeparableConv2D(256, (3,3), activation='relu', padding='same'))
# model.add(SeparableConv2D(256, (3,3), activation='relu', padding='same'))
# # model.add(BatchNormalization())
# model.add(MaxPooling2D(pool_size=(2, 2), strides=(2, 2), padding="SAME"))

# model.add(SeparableConv2D(512, (3,3), activation='relu', padding='same'))
# model.add(SeparableConv2D(512, (3,3), activation='relu', padding='same'))
# model.add(SeparableConv2D(512, (3,3), activation='relu', padding='same'))
# # model.add(BatchNormalization())
# model.add(MaxPooling2D(pool_size=(2, 2), strides=(2, 2), padding="SAME"))

# model.add(SeparableConv2D(512, (3,3), activation='relu', padding='same'))
# model.add(SeparableConv2D(512, (3,3), activation='relu', padding='same'))
# model.add(SeparableConv2D(512, (3,3), activation='relu', padding='same'))
# # model.add(BatchNormalization())
# model.add(MaxPooling2D(pool_size=(2, 2), strides=(2, 2), padding="SAME"))

# model.add(Flatten())
# model.add(Dense(4096, activation='relu'))
# model.add(Dense(4096, activation='relu'))
# # model.add(Dropout(0.7))
# model.add(Dense(1000, activation='relu'))
# # model.add(Dropout(0.5))

# model.add(Dense(2, activation='softmax'))

In [0]:
model.summary()

In [0]:
gen = ImageDataGenerator()
train_batches = gen.flow_from_directory("chest_xray/chest_xray/train", model.input_shape[1:3], color_mode="grayscale", shuffle=True, seed=1, batch_size=16)
valid_batches = gen.flow_from_directory("chest_xray/chest_xray/val",   model.input_shape[1:3], color_mode="grayscale", shuffle=True, seed=1, batch_size=16)
test_batches  = gen.flow_from_directory("chest_xray/chest_xray/test",  model.input_shape[1:3], color_mode="grayscale", shuffle=False, batch_size=8)

## **Experiment 1.** learning rate = 0.01

In [0]:
model.compile(Adam(lr=0.001),loss="categorical_crossentropy", metrics=["accuracy"])
model.fit_generator(train_batches, validation_data=valid_batches, epochs=3)

In [0]:
p = model.predict_generator(test_batches, verbose=True)
pre = pd.DataFrame(p)
pre["filename"] = test_batches.filenames
pre["label"] = (pre["filename"].str.contains("PNEUMONIA")).apply(int)
pre['pre'] = (pre[1]>0.5).apply(int)
print(pre)

In [0]:
recall_score(pre["label"], pre["pre"])

In [0]:
roc_auc_score(pre["label"],pre[1])

In [0]:
tpr,fpr,thres = roc_curve(pre["label"],pre[1])
roc = pd.DataFrame([tpr,fpr]).T
roc.plot(x=0,y=1)

## **Experiment 2.** learning rate = 0.001

In [0]:
model.compile(Adam(lr=0.001),loss="categorical_crossentropy", metrics=["accuracy"])
model.fit_generator(train_batches, validation_data=valid_batches, epochs=3)

In [0]:
p = model.predict_generator(test_batches, verbose=True)
pre = pd.DataFrame(p)
pre["filename"] = test_batches.filenames
pre["label"] = (pre["filename"].str.contains("PNEUMONIA")).apply(int)
pre['pre'] = (pre[1]>0.5).apply(int)
print(pre)

In [0]:
recall_score(pre["label"], pre["pre"])

In [0]:
roc_auc_score(pre["label"],pre[1])

In [0]:
tpr,fpr,thres = roc_curve(pre["label"],pre[1])
roc = pd.DataFrame([tpr,fpr]).T
roc.plot(x=0,y=1)

## **Experiment 3.** learning rate = 0.0001

In [0]:
model.compile(Adam(lr=0.0001), loss="categorical_crossentropy", metrics=["accuracy"])
model.fit_generator(train_batches, validation_data=valid_batches, epochs=3)

In [0]:
p = model.predict_generator(test_batches, verbose=True)
pre = pd.DataFrame(p)
pre["filename"] = test_batches.filenames
pre["label"] = (pre["filename"].str.contains("PNEUMONIA")).apply(int)
pre['pre'] = (pre[1]>0.5).apply(int)

In [0]:
recall_score(pre["label"],pre["pre"])

In [0]:
roc_auc_score(pre["label"],pre[1])

In [0]:
tpr,fpr,thres = roc_curve(pre["label"],pre[1])
roc = pd.DataFrame([tpr,fpr]).T
roc.plot(x=0,y=1)